In [ ]:
from ctapipe.io import EventSource
from astropy.table import Table, vstack
from ctapipe.visualization import CameraDisplay
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u


In [ ]:
fits_file = '/Users/vdk/muons2024/lapalma_simulations/simtel/zenith0/muon_table_run101.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.9
ring_containment_cut = 0.9
event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]))

In [ ]:
df_good_data

In [ ]:
filename = '/Users/vdk/muons2024/lapalma_simulations/simtel/zenith0/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 5000)
event_iterator = iter(source)
i = 0
events = np.array([])
for event in event_iterator:
    if event.index.event_id in event_ids:
        events = np.append(events, event)
        


#### Calculating pixel area and other geom things

In [ ]:
telescope = source.subarray.tel[1]
geometry = telescope.camera.geometry
optics = telescope.optics

focal_length = optics.equivalent_focal_length.to_value(u.m)
pixel_area = geometry.pix_area[0].to_value(u.m**2)
pixel_radius = np.sqrt(pixel_area / np.pi) / focal_length

mirror_radius = np.sqrt(optics.mirror_area.to_value(u.m**2) / np.pi)

In [ ]:
pixel_radius

#### True images of simulated events

In [ ]:
for i,event in enumerate(events[:10]):
        plt.figure(figsize = (12,12))
        camgeom = source.subarray.tel[1].camera.geometry
        #title="CT{}, run {} cog {} fit {} width {}".format(1,event.index.obs_id, round(event.dl1.tel[1].parameters.concentration.cog, 3), round(j.muon.tel[1].efficiency.optical_efficiency, 3), round(j.muon.tel[1].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title='1')
        disp.image = event.simulation.tel[1].true_image #.sum(axis=1)
        #disp.image = j.r0.tel[1].waveform[0].sum(axis=1)        
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        #plt.savefig(f"/Users/vdk/TrurImageCheckCOG{i}_COG=0_fitWrong")
        plt.show()

### R1 Distributions

#### R1 amplitudes per event

In [ ]:
x_value = np.arange(1,41)
plt.figure(figsize = (15,9))
for event_instance in events[:100]:
    plt.plot(x_value,event_instance.r1.tel[1].waveform[:,:,:].sum(axis=1).reshape(40), drawstyle='steps')

plt.grid(alpha = 0.5)
plt.xlabel("Time bin (ns)")
plt.ylabel("Sum of p.e. pver all pixels in each time bin")


In [ ]:
plt.figure(figsize = (15,9))
for pix_id in range(0,1855):
    plt.plot(events[1].r1.tel[1].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')

plt.grid(alpha = 0.5)
plt.xlabel("Time bin (ns)")
plt.ylabel("p.e./ns for each pixel")

#### Number of events that have pixels on the image with R1 amplitude > 2.5 p.e.


In [ ]:
pixel_numbers = []
for event_instance in events:
    #pixel_numbers.append((event_instance.r1.tel[1].waveform[0,:,:][event_instance.r1.tel[1].waveform[0].sum(axis=1) > 2.5]).shape[0])
    condition_mask = event_instance.r1.tel[1].waveform[0,:,:] > 2.5
    rows_with_condition = np.any(condition_mask, axis=1)
    pixel_numbers.append(np.sum(rows_with_condition))

In [ ]:
plt.hist(pixel_numbers, bins = 20)
plt.grid(alpha=0.5)

In [ ]:
condition_mask = event_instance.r1.tel[1].waveform[0,:,:] > 2.5
rows_with_condition = np.any(condition_mask, axis=1)
count = np.sum(rows_with_condition)
count

In [ ]:
(events[0].r1.tel[1].waveform[0,:,:][events[0].r1.tel[1].waveform[0].sum(axis=1) > 2.5]).shape[0]

#### Distribution of mean for variance per pixel

#### Distribution of time of max value R1 amplitude per pixel